# Resumen anual

In [1]:
import os
import pandas as pd


In [2]:
fecha_ini = '1981-01-01'
fecha_fin = '2020-12-31'


'''leer los archivos en un carpeta dada SI son archivos CSV'''
def leer_dir(path, cond1 = 'indice'):
    with os.scandir(path) as ficheros:
        ficheros = [fichero.name for fichero in ficheros if fichero.is_file() 
                                                            and fichero.name.endswith('12.csv') 
                                                            and fichero.name.count(cond1)>0]
    ficheros.sort()
    return ficheros

def procesar(nom):
    
    if (nom.count('indices') > 0):
        ruta = '../datos/3.0.spi_spei/'
    else:
        ruta = '../datos/1.0.variables/'   
    data = pd.read_csv(ruta + nom)
    
    data['fecha'] = pd.to_datetime(data['fecha'])
    data = data[(data['fecha'] >= fecha_ini) & (data['fecha'] <= fecha_fin)]
    
    '''Para variable precipitación se obtiene la media de la suma de todos los meses'''
    if (nom.count('pp') > 0): 
        '''Para precipitación'''
        return data.groupby(pd.Grouper(key='fecha', freq = 'YS')).sum()
    elif (nom.count('indice') > 0) :
        '''Para spi-spei'''
        return data.groupby(pd.Grouper(key='fecha', freq = 'YS')).mean()
    else:
        '''Para temperatura y humedad'''
        return data.groupby(pd.Grouper(key='fecha', freq = 'YS')).mean()


def adicionar_serie(s, df, col):
    df.sort_values(by='id', inplace=True, ascending=True)
    s.sort_index(ascending=True, inplace=True)
    c = nombre_col(col)
    df[c] = s.values
    return df
    
def nombre_col(cadena):
    cadena = cadena.split('.')
    cadena = cadena[0].split('_')
    cad = cadena[2] + '_media_anual'
    return cad

In [3]:
dir_est = '../datos/1.0.Estaciones/estaciones_senamhi_generado.csv'
cols = ['id', 'lon', 'lat', 'altitud', 'estacion']

estaciones = pd.read_csv(dir_est, usecols=cols)[cols]


## Generar resumen media anual en base a los datos CHIRPS, NASA y OBSERVADOS

In [4]:
fuente_archivos = ['chirps_mensual_pp_bc.csv','nasa_mensual_hmed.csv',
                   'nasa_mensual_tmax_bc.csv','nasa_mensual_tmed.csv','nasa_mensual_tmin_bc.csv',
                   'indices_spei1.csv','indices_spi1.csv',
                   'indices_spei3.csv','indices_spi3.csv',
                   'indices_spei6.csv','indices_spi6.csv',
                   'indices_spei12.csv','indices_spi12.csv']

for archivo in fuente_archivos:
    serie = procesar(archivo)     
    serie['fecha']=pd.DatetimeIndex(serie.index)
    serie['anio']=pd.DatetimeIndex(serie['fecha']).year
    serie.drop('fecha', axis=1, inplace=True)
    
    if (archivo.count('chirps_mensual'))>0:
        archivo = archivo.replace('chirps_mensual_','')
    elif (archivo.count('nasa_mensual'))>0:
        archivo = archivo.replace('nasa_mensual_','')
    else:
        archivo = archivo.replace('indices_','')
    serie.to_csv('../datos/1.0.variables/resumen_anual_'+archivo, index = False)  


In [5]:
serie

,100,101,102,105,106,107,108,111,112,113,...,119,120,121,200,201,202,203,204,205,anio
fecha,,,,,,,,,,,,,,,,,,,,,
1981-01-01,1.123000,1.010000,1.000000,0.744000,0.928000,0.820000,0.461000,0.898000,1.156000,1.091000,...,0.984000,1.162000,0.929000,0.621000,1.240000,0.228000,0.994000,-0.160000,1.099000,1981
1982-01-01,0.584250,0.119917,0.276417,0.785667,0.199750,0.275083,0.015333,0.153750,0.291250,0.868250,...,0.335250,0.249833,0.498667,0.417333,0.765417,0.044750,0.804250,-0.359083,0.826583,1982
1983-01-01,-1.781333,-2.310667,-1.787833,-1.641167,-1.404833,-1.667000,-1.976417,-2.165333,-1.511167,-1.633083,...,-2.025917,-1.712250,-2.065583,-1.832333,-1.132750,-2.034583,-1.781000,-2.420417,-1.583917,1983
1984-01-01,1.377750,1.556917,1.633083,1.676583,2.405917,2.018083,1.213500,1.503500,2.326000,2.063750,...,2.191167,1.923417,2.480833,2.161000,1.920250,1.762333,2.524750,1.511583,1.396583,1984
1985-01-01,1.516583,0.913167,0.943167,1.312917,1.432000,1.571083,0.707000,0.704917,1.509583,0.936750,...,1.135083,1.065000,1.103000,1.287583,1.026417,0.950250,1.171500,0.940000,1.195083,1985
1986-01-01,1.027417,0.826000,1.040417,0.523250,1.177333,1.019917,0.807167,0.657167,1.224750,0.965083,...,0.529917,0.941083,1.155833,1.054750,0.854167,0.887750,1.265417,0.915750,0.672917,1986
1987-01-01,0.258250,0.670750,0.463750,0.327500,0.877667,0.545000,0.363500,0.770167,0.660250,0.421083,...,0.940167,0.454583,0.400417,0.266833,0.008583,-0.185333,0.360833,-0.277167,0.509750,1987
1988-01-01,-0.308667,-0.642750,-0.559000,-0.164000,-0.026417,-0.327417,-0.605333,-0.777333,-0.012917,-0.560750,...,-0.125000,-0.269500,-0.119167,-0.426417,-0.227333,-0.935500,-0.388500,-1.062750,0.021750,1988
1989-01-01,-1.345500,-1.127417,-1.489167,-1.359583,-1.254417,-1.374000,-0.784917,-1.129083,-0.803833,-1.304833,...,-1.506500,-1.271333,-0.749333,-0.053417,-0.827750,-0.289833,-0.560500,-0.403167,-0.965917,1989
